In [1]:
import os
from pathlib import Path
import json
import pandas as pd

import sys
sys.path.insert(0,str(Path(os.path.abspath('.')).parent.parent))
from library.solar_resource_api.request_data import HistoricalRequest

## read data

In [16]:
ticket_path = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/data_sites"
api_dir = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3275_use_solar_resource_api/solar_api"
clearsky_dir = Path.home()/"OneDrive-3E/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3275_use_solar_resource_api/clearsky"

In [8]:
sites_metadata = pd.read_csv(os.path.join(ticket_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


# Request 

## solar_api

In [2]:
SOLAR_RESOURCE_KEY = os.environ["EEE_SOLAR_API_LEGACY_KEY"]

In [3]:
solar_resource_url = "https://api.3elabs.eu/solardata?"

In [12]:
for index, row in sites_metadata.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    historical_request = HistoricalRequest(
        latitude=latitude,
        longitude=longitude,
        variables=["global_horizontal"],
        resolution="h",
        start="2022-01-01",
        end="2024-01-01"
    )

    print(f"Request for site {index} start")
    prod_results_json, execution_time = historical_request.legacy_post_and_get_results(
        token=SOLAR_RESOURCE_KEY,
        url=solar_resource_url
    )
    print(f"Request for site {index} completed")

    data = pd.DataFrame(prod_results_json["data"], index=pd.DatetimeIndex(prod_results_json["index"]), columns=prod_results_json["columns"])

    file_name = f"{index}.csv"
    data.to_csv(os.path.join(output_dir, file_name))
    print(f"Data saved to {file_name}")


Request for site camborne start
Request for site camborne completed
Data saved to camborne.csv
Request for site carpentras start
Request for site carpentras completed
Data saved to carpentras.csv
Request for site budapest_lorinc start
Request for site budapest_lorinc completed
Data saved to budapest_lorinc.csv
Request for site magurele_mars start
Request for site magurele_mars completed
Data saved to magurele_mars.csv
Request for site cabauw start
Request for site cabauw completed
Data saved to cabauw.csv
Request for site cener start
Request for site cener completed
Data saved to cener.csv
Request for site payerne start
Request for site payerne completed
Data saved to payerne.csv
Request for site heino start
Request for site heino completed
Data saved to heino.csv
Request for site lauwersoog start
Request for site lauwersoog completed
Data saved to lauwersoog.csv
Request for site lelystad_locatie_23t start
Request for site lelystad_locatie_23t completed
Data saved to lelystad_locatie_2

## clearsky

In [15]:
times = pd.date_range(start="2022-01-01", end="2024-01-01", freq="H")


In [18]:
for index, metadata in sites_metadata.iterrows():
    location = Location(latitude=metadata["latitude"], longitude=metadata["longitude"], altitude=metadata["altitude"])
    clearsky = location.get_irre_clearsky(times=times)
    ghi = clearsky["ghi"].rename('global_horizontal')
    
    file_name = f"{index}.csv"  
    ghi.to_csv(os.path.join(output_dir, file_name))
    print(f"Data saved to {file_name}")
    

Data saved to camborne.csv
Data saved to carpentras.csv
Data saved to budapest_lorinc.csv
Data saved to magurele_mars.csv
Data saved to cabauw.csv
Data saved to cener.csv
Data saved to payerne.csv
Data saved to heino.csv
Data saved to lauwersoog.csv
Data saved to lelystad_locatie_23t.csv
Data saved to stavoren.csv
Data saved to herwijnen.csv
Data saved to hohenpeißenberg.csv
Data saved to mannheim.csv
Data saved to schleswig.csv
Data saved to görlitz.csv
Data saved to hamburg_fuhlsbüttel.csv
Data saved to seehausen.csv
Data saved to rostock_warnemünde.csv
Data saved to dresden_klotzsche.csv
Data saved to lautertal_oberlauter.csv
Data saved to leinefelde.csv
Data saved to bremen.csv
Data saved to stuttgart_schnarrenberg.csv
Data saved to nürnberg.csv
Data saved to weihenstephan_dürnast.csv
Data saved to saarbrücken_ensheim.csv
Data saved to aberporth.csv
Data saved to dunstaffnage.csv
Data saved to belfast_aldergrove_arpt.csv
Data saved to edinburgh.csv
Data saved to wattisham.csv
Data 

# Sum

## solar_api

In [21]:
site_api_sum = {}

for index, metadata in sites_metadata.iterrows():
    filename = f"{index}.csv"  
    ghi_data = pd.read_csv(os.path.join(api_dir, filename), index_col=0)
    api_sum = ghi_data["global_horizontal"].sum()
    
    site_api_sum[index] = api_sum


In [15]:
site_api_sum

{'camborne': 2389215.69,
 'carpentras': 3296460.8000000003,
 'budapest_lorinc': 2704020.6700000004,
 'magurele_mars': 2959101.36,
 'cabauw': 2357746.19,
 'cener': 3064857.71,
 'payerne': 2841512.15,
 'heino': 2293113.36,
 'lauwersoog': 2275081.73,
 'lelystad_locatie_23t': 2315635.98,
 'stavoren': 2331514.08,
 'herwijnen': 2349356.1600000006,
 'hohenpeißenberg': 2631284.4699999997,
 'mannheim': 2443531.02,
 'schleswig': 2169956.44,
 'görlitz': 2332862.9000000004,
 'hamburg_fuhlsbüttel': 2181907.43,
 'seehausen': 2275409.31,
 'rostock_warnemünde': 2305333.67,
 'dresden_klotzsche': 2319181.45,
 'lautertal_oberlauter': 2406957.2600000002,
 'leinefelde': 2307261.75,
 'bremen': 2230781.16,
 'stuttgart_schnarrenberg': 2508385.56,
 'nürnberg': 2431641.47,
 'weihenstephan_dürnast': 2553098.99,
 'saarbrücken_ensheim': 2485643.06,
 'aberporth': 2185180.2,
 'dunstaffnage': 1692810.96,
 'belfast_aldergrove_arpt': 1914921.95,
 'edinburgh': 1930749.5899999999,
 'wattisham': 2279769.64,
 'odiham': 225

In [14]:
site_name = "camborne"
site_api = site_api_sum.get(site_name, None)
if site_api is not None:
    print(f"The total ghi of solar_api for {site_name} is: {site_api}")
else:
    print(f"No data found for {site_name}")

The total clearsky for camborne is: 2389215.69


## clearsky

In [17]:
site_clearsky_sum = {}

for index, metadata in sites_metadata.iterrows():
    filename = f"{index}.csv"  
    ghi_data = pd.read_csv(os.path.join(clearsky_dir, filename), index_col=0)
    clearsky_sum = ghi_data["global_horizontal"].sum()
    
    site_clearsky_sum[index] = clearsky_sum


In [18]:
site_clearsky_sum

{'camborne': 3487416.37777132,
 'carpentras': 3891766.4264785224,
 'budapest_lorinc': 3675276.223671596,
 'magurele_mars': 3875644.613248259,
 'cabauw': 3366091.154643707,
 'cener': 3985886.3701683455,
 'payerne': 3727382.873076753,
 'heino': 3335127.804411168,
 'lauwersoog': 3268325.816792712,
 'lelystad_locatie_23t': 3332741.5624131532,
 'stavoren': 3302876.2963540065,
 'herwijnen': 3373633.5508863223,
 'hohenpeißenberg': 3679046.370754786,
 'mannheim': 3535553.4611858614,
 'schleswig': 3193440.1849560807,
 'görlitz': 3428055.652113712,
 'hamburg_fuhlsbüttel': 3253685.4326722356,
 'seehausen': 3304245.0392073425,
 'rostock_warnemünde': 3216266.827245507,
 'dresden_klotzsche': 3429797.1850525034,
 'lautertal_oberlauter': 3489491.2171579073,
 'leinefelde': 3416930.704051336,
 'bremen': 3292961.610639697,
 'stuttgart_schnarrenberg': 3588136.0875696484,
 'nürnberg': 3543008.5280303145,
 'weihenstephan_dürnast': 3621973.775991522,
 'saarbrücken_ensheim': 3563036.836656422,
 'aberporth': 3

In [19]:
site_name = "camborne"
site_clearsky = site_clearsky_sum.get(site_name, None)
if site_clearsky is not None:
    print(f"The total ghi of clearsky for {site_name} is: {site_clearsky}")
else:
    print(f"No data found for {site_name}")

The total ghi of clearsky for camborne is: 3487416.37777132


# Divide

In [29]:
for index, metadata in sites_metadata.iterrows():
    site_name = index
    api_sum = site_api_sum.get(site_name, None)
    clearsky_sum = site_clearsky_sum.get(site_name, None)
    
    if api_sum is not None and clearsky_sum is not None and clearsky_sum != 0:
        clearness = (api_sum / clearsky_sum) * 100
    else:
        clearness = None
    
    sites_metadata.loc[index, "clearness (%)"] = clearness


In [30]:
sites_metadata


,country,latitude,longitude,altitude,provider,clearness (%)
station_name,,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn,68.509619
carpentras,France,44.08,5.06,100.0,bsrn,84.703460
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn,73.573264
magurele_mars,Romania,44.34,26.01,110.0,bsrn,76.351205
cabauw,Netherlands,51.97,4.93,0.0,bsrn,70.044039
...,...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc,64.296801
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc,71.090747
gonzaga,Italy,44.96,10.77,16.0,lombardy,78.808519
